# Проект "Как изменилась тематика фильмов за последние 20 лет"    
## Парсинг.
*Источник данных - неофициальный API Кинопоиска https://kinopoisk.dev/*  
*Сравниваемые периоды: 2020-2025 гг. и 2000-2005 гг.*

*API позволяет загружать по 1 странице, состоящей из списка 250 фильмов с описанием.*

Для анализа отберу фильмы по таким фильтрам:
- по количеству оценок в порядке убывания, чтобы отобрать наиболее популярные фильмы;
- поставлю фильтр, что это должен быть именно фильм, а не мультфильм, сериал и т.п.;
- отмечу, что описание фильма != 'Null';
- а также отмечу период 2020-2025гг, а для второго корпуса 2000-2005гг.

*Просмотрев результаты поиска, я решила взять первые 50 страниц поисковой выдачи.
Должно получиться около 12 500 фильмов в каждом корпусе.*

In [4]:
import requests
import re

In [5]:
# API позволяет загружать по 1 странице, состоящей из списка 250 фильмов с описанием.
# изначально я получила такой код для загрузки первой страницы поисковой выдачи:

url = "https://api.kinopoisk.dev/v1.4/movie?page=1&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2020-2025"

headers = {
    "accept": "application/json",
    "X-API-KEY": "AW8KS88-T27M50N-P61YKCP-1GSXSJX"
}

response = requests.get(url, headers=headers)

response = response.text

In [6]:
# посмотрим, в каком виде мы получаем страницы:
response

'{"docs":[{"id":1318972,"name":"Гнев человеческий","alternativeName":"Wrath of Man","enName":null,"names":[{"name":"Cash Truck","language":"US","type":"working title"},{"name":"运钞车","language":"CN","type":null},{"name":"Un homme en colère","language":"FR","type":null},{"name":"Infiltrado","language":"BR","type":null},{"name":"Гнев человеческий","language":"RU","type":null},{"name":"Justicia Implacable","language":"MX","type":null},{"name":"คนคลั่งแค้น ปล้นผ่านรก","language":"TH","type":null},{"name":"캐시트럭","language":"KR","type":null},{"name":"İntikam Vakti","language":"TR","type":null},{"name":"Para Kamyonu","language":"TR","type":null},{"name":"Cash Truck","language":"NL","type":null},{"name":"La furia di un uomo","language":"IT","type":null},{"name":"玩命鈔劫","language":"HK","type":null},{"name":"خشم مرد","language":"IR","type":null},{"name":"キャッシュトラック：2021","language":"JP","type":null},{"name":"玩命鈔劫","language":"TW","type":null},{"name":"Wrath of Man","language":"KR","type":"Daum Movi

In [7]:
# для проекта возьмём первые 50 страниц поисковой выдачи.
# для этого сгенерируем ссылки на них в цикле и сохраним в links_1_period и links_2_period:
links_1_period = []
links_2_period = []

for i in range(1, 51):
    links_1_period.append(f"https://api.kinopoisk.dev/v1.4/movie?page={str(i)}&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2020-2025")
    links_2_period.append(f"https://api.kinopoisk.dev/v1.4/movie?page={str(i)}&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2000-2005")

links_2_period[:3]

['https://api.kinopoisk.dev/v1.4/movie?page=1&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2000-2005',
 'https://api.kinopoisk.dev/v1.4/movie?page=2&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2000-2005',
 'https://api.kinopoisk.dev/v1.4/movie?page=3&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2000-2005']

In [78]:
# а теперь загрузим все эти странички:
def load_pages(links):
    pages = []
    
    for link in links:
        page = requests.get(link, headers=headers).text
        pages.append(page)
    return(pages)

In [80]:
pages_1 = load_pages(links_1_period)
pages_2 = load_pages(links_2_period)
pages_2[-1]

'{"message":"Вы израсходовали ваш суточный лимит по запросам. Чтобы получить больше запросов, обновите тариф в боте @kinopoiskdev_bot\'","error":"Forbidden","statusCode":403}'

In [72]:
# достанем описание к каждому фильму и сохраним его в переменную с помощью функции:
def collect_descriptions(pages_list):
    all_descriptions = []
    
    for page in pages_list:
        page = re.sub(r'(\xa0|\\n|\\r)', ' ', page)  # сразу заменим неразрывные пробелы и переносы строк на пробел
        descriptions = re.findall(r'"description":"([А-ЯЁA-Za-zа-яё].+?)","', page)  # среди описаний всё-таки есть пустые, не отмеченные как 'Null'
        # поэтому в регулярном выражении укажем, что нас интересуют описания, начинающиеся с буквы.
       
        all_descriptions.append(descriptions)
        return(all_descriptions)

In [76]:
all_descriptions_1 = collect_descriptions(pages_1)
all_descriptions_2 = collect_descriptions(pages_2)
all_descriptions_2[:1]

[['Участвуя в программе на телевидении, Данила Багров встречает своих друзей по службе в Чечне. Одного из них внезапно убивают. Выясняется, что у того были неприятности из-за брата-хоккеиста в Америке. Данила должен разобраться. Он вылетает в Америку и за компанию берёт с собой старшего брата.',
  'Жизнь десятилетнего Гарри Поттера нельзя назвать сладкой: родители умерли, едва ему исполнился год, а от дяди и тёти, взявших сироту на воспитание, достаются лишь тычки да подзатыльники. Но в одиннадцатый день рождения Гарри всё меняется. Странный гость, неожиданно появившийся на пороге, приносит письмо, из которого мальчик узнаёт, что на самом деле он - волшебник и зачислен в школу магии под названием Хогвартс. А уже через пару недель Гарри будет мчаться в поезде Хогвартс-экспресс навстречу новой жизни, где его ждут невероятные приключения, верные друзья и самое главное — ключ к разгадке тайны смерти его родителей.',
  'Жизнь харизматичного авантюриста, капитана Джека Воробья, полная увлека

In [36]:
# оценим объём получившегося корпуса:
def mesure_volume(all_descriptions):
    films_base = 0  # здесь посчитаем количество фильмов в коллекции
    corpus_volume = 0  # а здесь - количество токенов
    
    for descriptions_list in all_descriptions:
        #print(len(i))  # можно посмотреть, сколько фильмов с каждой странички вошли в корпус
        films_base += len(descriptions_list)
        for description in descriptions_list:
            corpus_volume += len(description.split())
    return(films_base, corpus_volume)

In [44]:
mesure_volume(all_descriptions_1), mesure_volume(all_descriptions_2)
# итак, получилось 2 корпуса: первый объёмом 480 266 токенов и второй объёмом 719 381 токен. В обоих корпусах около 12 000 фильмов.

((11837, 480266), (11981, 719381))

In [56]:
# прербразуем для удобства получившиеся списки списков в плоские списки:
def create_flat_list(list):
    flat_list = sum(list, [])
    return(flat_list)

In [ ]:
all_descriptions_1 = create_flat_list(all_descriptions_1)
all_descriptions_2 = create_flat_list(all_descriptions_2)

In [54]:
# проверим, что получилось:
all_descriptions_1[:5]

['Грузовики лос-анджелесской инкассаторской компании Fortico Security часто подвергаются нападениям, и во время очередного ограбления погибают оба охранника. Через некоторое время в компанию устраивается крепкий немногословный британец Патрик Хилл. Он получает от босса прозвище Эйч и, впритык к необходимому минимуму пройдя тесты по фитнесу, стрельбе и вождению, отправляется на первое задание. Вскоре и его грузовик пытаются ограбить вооруженные налётчики, но Эйч в одиночку расправляется с целой бандой и становится героем. Кажется, слава и уважение коллег его совершенно не интересуют, ведь он преследует свои цели.',
 'Гриша, бывший мажор, побывавший холопом и ставший человеком, после путешествия в «прошлое» чутко реагирует на любую несправедливость. И, конечно, не может пройти мимо беспредела, который творит наглая и избалованная Катя. Ничего удивительного, что вскоре мажорка обнаруживает себя в другом времени.',
 'Если ты идешь на рыбалку — будь готов к тому, что вытянешь рыбу своей меч

In [68]:
# соберём датафрейм из полученных данных и далее сохраним в файл:
import pandas as pd
df_1 = pd.DataFrame(all_descriptions_1, columns=['Описание фильма'])
df_1.head()   

,Описание фильма
0,Грузовики лос-анджелесской инкассаторской комп...
1,"Гриша, бывший мажор, побывавший холопом и став..."
2,"Если ты идешь на рыбалку — будь готов к тому, ..."
3,"Афганистан, март 2018 года. Во время спецопера..."
4,История о путешествии взрослого героя к своему...


In [70]:
df_2 = pd.DataFrame(all_descriptions_2, columns=['Описание фильма'])
df_2.head()   

,Описание фильма
0,"Участвуя в программе на телевидении, Данила Ба..."
1,Жизнь десятилетнего Гарри Поттера нельзя назва...
2,"Жизнь харизматичного авантюриста, капитана Дже..."
3,"Фрэнк Эбегнейл успел поработать врачом, адвока..."
4,Римская империя. Бесстрашного и благородного г...


In [66]:
df_1.to_excel('Descriptions_1.xlsx')
df_2.to_excel('Descriptions_2.xlsx')